### 쇼핑몰 크롤링 DB에 저장

In [62]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql

In [63]:
def conn(d_name) :
 import pymysql
 host_name = 'localhost'
 host_port = 3306
 username = 'root'
 password = 'gPwls03!5'
 database_name = d_name
 db = pymysql.connect(
 host=host_name, # MySQL Server Address
 port=host_port, # MySQL Server Port
 user=username, # MySQL username
 passwd=password, # password for MySQL username
 db=database_name, # Database name
 charset='utf8'
 )
 return db

In [64]:
db=conn('beauty_shop')
cursor=db.cursor()

In [65]:
sql = "show databases"
cursor.execute(sql)
result=cursor.fetchall()
result

(('beauty_shop',),
 ('ecommerce',),
 ('employees',),
 ('information_schema',),
 ('my_emp',),
 ('my_emp01',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sqldb',),
 ('student_mgmt',),
 ('students',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [66]:
sql="use beauty_shop"
cursor.execute(sql)

0

In [67]:
sql = "select database()"
cursor.execute(sql)
result=cursor.fetchone()
result

('beauty_shop',)

In [68]:
#table 생성
sql = '''
   
    CREATE TABLE product (
        PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE VARCHAR(100),
        DISCOUNT_PRICE VARCHAR(100),
        link VARCHAR(200),
        PRIMARY KEY(PRODUCT_CODE)
    );
'''
cursor.execute(sql)
db.commit()

OperationalError: (1050, "Table 'product' already exists")

In [69]:
sql="show tables"
cursor.execute(sql)
result=cursor.fetchall()
result

(('product',),)

In [70]:
sql = 'desc product'
cursor.execute(sql)
result=cursor.fetchall()
result

(('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

In [71]:
# 크롤링 문서 요청해서 응답객체 반환(첫번째 크롤링 문서 요청 후 응답)
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [72]:
# box안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로 반환하는 함수
# 데이터 전처리 : 제품명에 ' 제거/ 가격 USD 제거 / 세일가격 없는 경우 처리
def get_product_info(box) :
    strong_tag = box.find("strong",{"class":"name"})
    # 품목 추출
    span =strong_tag.text.split(':')[1]
 
    # 세부페이지링크 추출
    a = strong_tag.find("a")
    sub_link = 'https://jolse.com' + a["href"]
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = "'"+price_ul.findAll("span")[1].text+"'"
    sals_price = "'"+box.find('ul').findAll('span')[-1].text+"'"
 
    # 데이터 전처리
    title = span.replace("'","''") # ' 처리
    ord_price = price_span
    dis_price = sals_price
    ord_price = price_span[1].text.split(' ')[1] # USD 제거
    dis_price = price_span[-1].text.split(' ')[1]
  # 세일 가격이 없는 경우
    if dis_price =='' :
        dis_price = '0.0'
 
 
 # 최종 data 추출 후 반환
    return{"prd_name":title,"price":ord_price,"sale_price":dis_price,"sub_link":sub_link}



In [73]:
def save_data(prd_info) :
# print(prd_info)
 
 # insert 구문
     sql = "INSERT INTO product (title, ori_price, discount_price,link) values('" \
         + prd_info["prd_name"] \
         +"'," \
         + prd_info['price']\
         +"," \
         + prd_info['sale_price']\
         +",'"\
         + prd_info['sub_link']\
         + "');"
     print(sql)
     cursor.execute(sql)


In [74]:
# 전달된 url 페이지에 접근해서 해당페지의 전체 상품 데이터를 추출 한 후 각 상품마다 get_product_info()
# 각 상품에대한 추출 정보를 받아옴 - 들어온 각 상품 정보를 리스트에 저장 한 수 해당 반환
def get_page_products(url) :
     url=url
     html = urlopen(url)
     htmls = html.read()
     # print(htmls)
     bs_obj = BeautifulSoup(htmls,"html.parser")
     ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
     # ul class:prdList grid4
     ul=bs_obj.find("ul",{"class":"prdList grid5"})
     ## 품목 1개를 담고 있는 div 태그 추출
     ## div class:box
     prd_boxes = ul.findAll("div", {"class":"description"}) #1개 페이지의 전체 상품
     # 반환되는 품목 데이터를 db에 insert : 함수호출해서 진행
     for box in prd_boxes :
         prd = get_product_info(box)
         print(prd)
         save_data(prd) #사용자 정의 함수(생성해야 함)

In [75]:
from tqdm import tqdm_notebook # 상태바 표시
#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열
#last= int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])
last=3
for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집
# for i in range(1,2) : # 1페이지 insert
     # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    get_page_products(urlfin)

C:\Users\82105\AppData\Local\Temp\ipykernel_4568\4203702665.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집


  0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'text'

In [76]:
db.commit()


In [77]:
sql = "select * from product"
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml',
  'USD 22.00',
  'USD 17.60',
  'https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/'),
 (2,
  ' Cos De BAHA Centella Facial Toner 200ml',
  'USD 16.45',
  'USD 13.98',
  'https://jolse.com/product/cos-de-baha-centella-facial-toner-200ml/40098/category/1019/display/2/'),
 (3,
  ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  'USD 37.00',
  'USD 24.38',
  'https://jolse.com/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/'),
 (4,
  ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml',
  'USD 22.00',
  'USD 17.60',
  'https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/'),
 (5,
  ' Cos De BAHA Centella Facial Toner 200ml',
  'USD 16.45',
  'USD 13.98',
  'https://jolse.com/product/cos-de-baha-centella-facial-toner-200ml/40098/category/1019/display/2/

In [78]:
db.close()

In [79]:
db = conn('beauty_shop')

In [80]:
sql = "select * from product"
df = pd.read_sql(sql,db)
df

C:\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60,https://jolse.com/product/some-by-mi-propolis-...
1,2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98,https://jolse.com/product/cos-de-baha-centella...
2,3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 24.38,https://jolse.com/product/haruharu-wonder-blac...
3,4,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60,https://jolse.com/product/some-by-mi-propolis-...
4,5,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98,https://jolse.com/product/cos-de-baha-centella...
5,6,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 24.38,https://jolse.com/product/haruharu-wonder-blac...
6,7,SOME BY MI Propolis B5 Glow Barrier Calming T...,Price,USD 17.60,https://jolse.com/product/some-by-mi-propolis-...
7,8,Cos De BAHA Centella Facial Toner 200ml,Price,USD 13.98,https://jolse.com/product/cos-de-baha-centella...
8,9,Haruharu WONDER Black Rice Hyaluronic Toner 3...,Price,USD 24.38,https://jolse.com/product/haruharu-wonder-blac...
9,10,SOME BY MI Propolis B5 Glow Barrier Calming T...,Price,USD 17.60,https://jolse.com/product/some-by-mi-propolis-...


In [61]:
sql = "select title,ori_price,discount_price from product"
df = pd.read_sql(sql,db)
df

C:\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,ori_price,discount_price
0,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
1,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
2,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 24.38
3,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
4,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
5,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 24.38
6,SOME BY MI Propolis B5 Glow Barrier Calming T...,Price,USD 17.60
7,Cos De BAHA Centella Facial Toner 200ml,Price,USD 13.98
8,Haruharu WONDER Black Rice Hyaluronic Toner 3...,Price,USD 24.38
9,SOME BY MI Propolis B5 Glow Barrier Calming T...,Price,USD 17.60


### 정리

1. 데이터베이스 생성
2. mysql 데이터베이스 연결
3. 테이블 생성
4. 크롤링 문서 요청해서 응답객체 반환

5. 1개의 상품의 정보를 추출, 데이터 전처리
6. 5에서 insert 구문에 추가->5번 순서의 함수의 return 값을 받아옴
7. url에서 상품의 박스 부분?? 추출 5,6 함수 호출함
8. insert 가 제대로 됐는지..sql 문 작성하여 실행